In [6]:
import openai
from latex2sympy2 import latex2sympy
import json

#model = "gpt-3.5-turbo-0613"
model = "gpt-4-0613"

def calc_latex_math_expression(expression):
    sympy_expr = latex2sympy(expression)
    result = sympy_expr.doit()
    return str(result)

def ask_user(question):
    return input(question)

functions = [
     {
      "name": "calc_latex_math_expression",
      "description": "Calculate a Latex mathematical expression",
      "parameters": {
        "type": "object",
        "properties": {
          "expression": {
            "type": "string",
            "description": "The Latex expression to calculate"
          },
        },
        "required": ["expression"]
      }
    },
    {
      "name": "ask_user",
      "description": "Asks the user for input",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question to ask to the user"
          },
        },
        "required": ["question"]
      }
    },
  ]

available_functions = {
    "calc_latex_math_expression": calc_latex_math_expression,
    "ask_user": ask_user,
}

In [7]:
def run_conversation(content):
  messages = [
    {"role": "user", "content": content},
    ]
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  messages.append(response_message)  # extend conversation with assistant's reply

  # Step 2: check if GPT wanted to call a function
  while response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    if function_name not in available_functions:
      raise Exception("Function " + function_name + " not found")
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Step 4: send the info on the function call and function response to GPT
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    response_message = response["choices"][0]["message"]
    messages.append(response_message)  # extend conversation with assistant's reply

  for message in messages:
    if message.get("content"):
      print(message["role"] + ": " + message["content"])
    if message.get("function_call"):
      print(" -> calling function " + message["function_call"]["name"] + " with arguments " + json.dumps(message["function_call"]["arguments"]))
  

In [8]:
run_conversation("If my brother is 3 years younger than me, how old is my brother? Ask user for any missing information. Explain step by step.")

user: If my brother is 3 years younger than me, how old is my brother? Ask user for any missing information. Explain step by step.
assistant: Sure, in order to perform the calculation, I need to know your age. Let me ask for this.
 -> calling function ask_user with arguments "{\n  \"question\": \"What is your current age?\"\n}"
function: 34
assistant: Thank you for providing your age. Now we can proceed with the calculation. Here's the step-by-step process:

1. You mentioned that your brother is 3 years younger than you.
2. Your age is 34 years.
3. To find out your brother's age, we subtract 3 from your age.

So, the calculation will be: 34 - 3 = 31

Therefore, your brother is 31 years old.
